In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Bert/data.csv"
df = pd.read_csv(path)
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Bert/bert_withstopwordchangelevels_150_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,0,-0.579412,0.188265,0.858297,-0.639750,-0.502543,-0.139287,0.527588,0.270662,0.144507,...,-0.183825,-0.139413,-0.246331,0.518628,-0.393642,0.874948,0.515016,-1.261790,0.575274,-0.120876
1,1,0.921289,0.217535,-0.961071,-0.520875,-1.292083,-0.628445,-1.341185,-1.262405,-0.106842,...,1.457121,1.169449,1.294919,-1.527045,1.830662,-1.629801,1.507712,-0.745857,-1.024986,1.705361
2,2,0.166176,0.464150,-0.482993,-0.359226,-1.121885,-0.649730,-1.583161,-0.746098,-0.827201,...,0.569310,0.552475,0.819384,-0.796547,0.398506,-0.151474,0.589110,-2.190126,-1.242226,0.947673
3,3,-0.843833,-0.810981,-0.171127,0.764811,0.706509,0.237682,0.871839,1.161460,0.500884,...,-1.067293,-1.048949,0.190678,0.249998,-0.491930,0.395937,-0.818597,1.010062,1.200272,-0.820093
4,4,1.554764,1.548400,-2.519800,-1.189661,-0.770486,-0.156267,-0.783727,-2.912629,-0.592351,...,2.126538,1.024326,2.182677,-1.497850,1.432716,-2.120613,1.263199,0.019320,-1.764692,1.381868


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-0.579412,0.188265,0.858297,-0.639750,-0.502543,-0.139287,0.527588,0.270662,0.144507,-0.822466,...,-0.183825,-0.139413,-0.246331,0.518628,-0.393642,0.874948,0.515016,-1.261790,0.575274,-0.120876
1,0.921289,0.217535,-0.961071,-0.520875,-1.292083,-0.628445,-1.341185,-1.262405,-0.106842,1.526470,...,1.457121,1.169449,1.294919,-1.527045,1.830662,-1.629801,1.507712,-0.745857,-1.024986,1.705361
2,0.166176,0.464150,-0.482993,-0.359226,-1.121885,-0.649730,-1.583161,-0.746098,-0.827201,0.788077,...,0.569310,0.552475,0.819384,-0.796547,0.398506,-0.151474,0.589110,-2.190126,-1.242226,0.947673
3,-0.843833,-0.810981,-0.171127,0.764811,0.706509,0.237682,0.871839,1.161460,0.500884,-0.939422,...,-1.067293,-1.048949,0.190678,0.249998,-0.491930,0.395937,-0.818597,1.010062,1.200272,-0.820093
4,1.554764,1.548400,-2.519800,-1.189661,-0.770486,-0.156267,-0.783727,-2.912629,-0.592351,1.375486,...,2.126538,1.024326,2.182677,-1.497850,1.432716,-2.120613,1.263199,0.019320,-1.764692,1.381868


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 150), (524, 150))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3599511214802623


**Decision Tree**

In [15]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [16]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [17]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [18]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [19]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [20]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.469672,0.469672,0.476406,0.452243,0.379771,0.379771,0.354171,0.356699


**Random Forest**

In [21]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [22]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [23]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [24]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.4198    0.2166    0.2857       157
           2     0.4691    0.8017    0.5919       237
           3     0.2368    0.0692    0.1071       130

    accuracy                         0.4447       524
   macro avg     0.3752    0.3625    0.3283       524
weighted avg     0.3967    0.4447    0.3799       524



In [25]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.469672,0.469672,0.476406,0.452243,0.379771,0.379771,0.354171,0.356699
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.444656,0.444656,0.396710,0.379897


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6338    0.3740    0.4704       361
           2     0.5249    0.8192    0.6398       553
           3     0.6458    0.3039    0.4133       306

    accuracy                         0.5582      1220
   macro avg     0.6015    0.4990    0.5078      1220
weighted avg     0.5875    0.5582    0.5329      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3012    0.1592    0.2083       157
           2     0.4412    0.6962    0.5401       237
           3     0.2090    0.1077    0.1421       130

    accuracy                         0.3893       524
   macro avg     0.3171    0.3210    0.2969       524
weighted avg     0.3416    0.3893    0.3420       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.482787,0.482787,0.713185,0.344373,0.454198,0.454198,0.394719,0.310403
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.410305,0.410305,0.358535,0.356639
2,Adaboost,0.558197,0.558197,0.587463,0.532881,0.389313,0.389313,0.341626,0.341963


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9877    0.8864    0.9343       361
           2     0.8614    1.0000    0.9255       553
           3     0.9921    0.8235    0.9000       306

    accuracy                         0.9221      1220
   macro avg     0.9471    0.9033    0.9199      1220
weighted avg     0.9315    0.9221    0.9217      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3367    0.2102    0.2588       157
           2     0.4770    0.7426    0.5809       237
           3     0.2281    0.1000    0.1390       130

    accuracy                         0.4237       524
   macro avg     0.3473    0.3509    0.3262       524
weighted avg     0.3732    0.4237    0.3748       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.482787,0.482787,0.713185,0.344373,0.454198,0.454198,0.394719,0.310403
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.410305,0.410305,0.358535,0.356639
2,Adaboost,0.558197,0.558197,0.587463,0.532881,0.389313,0.389313,0.341626,0.341963
3,GBM,0.922131,0.922131,0.931534,0.921720,0.423664,0.423664,0.373201,0.374759


**SVM Model**

In [26]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [27]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [28]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [29]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4887    0.5402    0.5132       361
           2     0.6721    0.4485    0.5380       553
           3     0.4358    0.6438    0.5198       306

    accuracy                         0.5246      1220
   macro avg     0.5322    0.5441    0.5236      1220
weighted avg     0.5586    0.5246    0.5261      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3096    0.3885    0.3446       157
           2     0.4342    0.2785    0.3393       237
           3     0.2571    0.3462    0.2951       130

    accuracy                         0.3282       524
   macro avg     0.3337    0.3377    0.3263       524
weighted avg     0.3530    0.3282    0.3299       524



In [30]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.469672,0.469672,0.476406,0.452243,0.379771,0.379771,0.354171,0.356699
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.444656,0.444656,0.396710,0.379897
2,SVM,0.524590,0.524590,0.558574,0.526064,0.328244,0.328244,0.352959,0.329942


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4894    0.5125    0.5007       361
           2     0.5700    0.7143    0.6340       553
           3     0.4966    0.2418    0.3253       306

    accuracy                         0.5361      1220
   macro avg     0.5187    0.4895    0.4867      1220
weighted avg     0.5278    0.5361    0.5171      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3455    0.3631    0.3540       157
           2     0.4641    0.5992    0.5230       237
           3     0.1509    0.0615    0.0874       130

    accuracy                         0.3950       524
   macro avg     0.3202    0.3413    0.3215       524
weighted avg     0.3508    0.3950    0.3643       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.482787,0.482787,0.713185,0.344373,0.454198,0.454198,0.394719,0.310403
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.410305,0.410305,0.358535,0.356639
2,Adaboost,0.558197,0.558197,0.587463,0.532881,0.389313,0.389313,0.341626,0.341963
3,GBM,0.922131,0.922131,0.931534,0.921720,0.423664,0.423664,0.373201,0.374759
4,SVM,0.557377,0.557377,0.558512,0.526660,0.398855,0.398855,0.358660,0.364587
5,KNN,0.536066,0.536066,0.527750,0.517128,0.395038,0.395038,0.350839,0.364324


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3047    0.2161    0.2528       361
           2     0.5222    0.1700    0.2565       553
           3     0.2538    0.6503    0.3651       306

    accuracy                         0.3041      1220
   macro avg     0.3602    0.3455    0.2915      1220
weighted avg     0.3905    0.3041    0.2827      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2650    0.1975    0.2263       157
           2     0.4556    0.1730    0.2508       237
           3     0.2461    0.6000    0.3490       130

    accuracy                         0.2863       524
   macro avg     0.3222    0.3235    0.2753       524
weighted avg     0.3465    0.2863    0.2678       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.482787,0.482787,0.713185,0.344373,0.454198,0.454198,0.394719,0.310403
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.410305,0.410305,0.358535,0.356639
2,Adaboost,0.558197,0.558197,0.587463,0.532881,0.389313,0.389313,0.341626,0.341963
3,GBM,0.922131,0.922131,0.931534,0.921720,0.423664,0.423664,0.373201,0.374759
4,SVM,0.557377,0.557377,0.558512,0.526660,0.398855,0.398855,0.358660,0.364587
5,KNN,0.536066,0.536066,0.527750,0.517128,0.395038,0.395038,0.350839,0.364324
6,GNB,0.304098,0.304098,0.390534,0.282655,0.286260,0.286260,0.346474,0.267797
